In [ ]:
!pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ML TTS/Online_Retail.csv',index_col=0)

In [ ]:
df.shape

(541909, 8)

In [ ]:
df.head()

,Country,CustomerID,Description,InvoiceNo,Quantity,StockCode,UnitPrice,InvoiceDate_1
0,United Kingdom,17850.0,WHITE HANGING HEART T-LIGHT HOLDER,536365.0,6.0,85123A,2.55,2010-12-01 08:26:00
1,United Kingdom,17850.0,WHITE METAL LANTERN,536365.0,6.0,71053.0,3.39,2010-12-01 08:26:00
2,United Kingdom,17850.0,CREAM CUPID HEARTS COAT HANGER,536365.0,8.0,84406B,2.75,2010-12-01 08:26:00
3,United Kingdom,17850.0,KNITTED UNION FLAG HOT WATER BOTTLE,536365.0,6.0,84029G,3.39,2010-12-01 08:26:00
4,United Kingdom,17850.0,RED WOOLLY HOTTIE WHITE HEART.,536365.0,6.0,84029E,3.39,2010-12-01 08:26:00


In [ ]:
df.Description = df.Description.str.strip()

In [ ]:
df.InvoiceNo.sort_values() # C in invoice is data entry mistake that we need to fix

0         536365.0
1         536365.0
2         536365.0
3         536365.0
4         536365.0
            ...   
540449     C581490
541541     C581499
541715     C581568
541716     C581569
541717     C581569
Name: InvoiceNo, Length: 541909, dtype: object

In [ ]:
df[df.InvoiceNo.str.contains('C')]

,Country,CustomerID,Description,InvoiceNo,Quantity,StockCode,UnitPrice,InvoiceDate_1
141,United Kingdom,14527.0,Discount,C536379,-1.0,D,27.50,2010-12-01 09:41:00
154,United Kingdom,15311.0,SET OF 3 COLOURED FLYING DUCKS,C536383,-1.0,35004C,4.65,2010-12-01 09:49:00
235,United Kingdom,17548.0,PLASTERS IN TIN CIRCUS PARADE,C536391,-12.0,22556.0,1.65,2010-12-01 10:24:00
236,United Kingdom,17548.0,PACK OF 12 PINK PAISLEY TISSUES,C536391,-24.0,21984.0,0.29,2010-12-01 10:24:00
237,United Kingdom,17548.0,PACK OF 12 BLUE PAISLEY TISSUES,C536391,-24.0,21983.0,0.29,2010-12-01 10:24:00
...,...,...,...,...,...,...,...,...
540449,United Kingdom,14397.0,ZINC T-LIGHT HOLDER STARS SMALL,C581490,-11.0,23144.0,0.83,2011-12-09 09:57:00
541541,United Kingdom,15498.0,Manual,C581499,-1.0,M,224.69,2011-12-09 10:28:00
541715,United Kingdom,15311.0,VICTORIAN SEWING BOX LARGE,C581568,-5.0,21258.0,10.95,2011-12-09 11:57:00
541716,United Kingdom,17315.0,HANGING HEART JAR T-LIGHT HOLDER,C581569,-1.0,84978.0,1.25,2011-12-09 11:58:00


In [ ]:
def remove_c(val):
    if 'C' in val or 'A' in val:
        return float(val[1:])
    else:
        return float(val)

df.InvoiceNo = df.InvoiceNo.apply(remove_c)

In [ ]:
df

,Country,CustomerID,Description,InvoiceNo,Quantity,StockCode,UnitPrice,InvoiceDate_1
0,United Kingdom,17850.0,WHITE HANGING HEART T-LIGHT HOLDER,536365.0,6.0,85123A,2.55,2010-12-01 08:26:00
1,United Kingdom,17850.0,WHITE METAL LANTERN,536365.0,6.0,71053.0,3.39,2010-12-01 08:26:00
2,United Kingdom,17850.0,CREAM CUPID HEARTS COAT HANGER,536365.0,8.0,84406B,2.75,2010-12-01 08:26:00
3,United Kingdom,17850.0,KNITTED UNION FLAG HOT WATER BOTTLE,536365.0,6.0,84029G,3.39,2010-12-01 08:26:00
4,United Kingdom,17850.0,RED WOOLLY HOTTIE WHITE HEART.,536365.0,6.0,84029E,3.39,2010-12-01 08:26:00
...,...,...,...,...,...,...,...,...
541904,France,12680.0,PACK OF 20 SPACEBOY NAPKINS,581587.0,12.0,22613.0,0.85,2011-12-09 12:50:00
541905,France,12680.0,CHILDREN'S APRON DOLLY GIRL,581587.0,6.0,22899.0,2.10,2011-12-09 12:50:00
541906,France,12680.0,CHILDRENS CUTLERY DOLLY GIRL,581587.0,4.0,23254.0,4.15,2011-12-09 12:50:00
541907,France,12680.0,CHILDRENS CUTLERY CIRCUS PARADE,581587.0,4.0,23255.0,4.15,2011-12-09 12:50:00


In [ ]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
francedf = df[df.Country=='United Kingdom'].groupby(['InvoiceNo','Description'])['Quantity'].sum()

In [ ]:
francedf

InvoiceNo  Description                        
536365.0   CREAM CUPID HEARTS COAT HANGER          8.0
           GLASS STAR FROSTED T-LIGHT HOLDER       6.0
           KNITTED UNION FLAG HOT WATER BOTTLE     6.0
           RED WOOLLY HOTTIE WHITE HEART.          6.0
           SET 7 BABUSHKA NESTING BOXES            2.0
                                                  ... 
581585.0   ZINC WILLIE WINKIE  CANDLE STICK       24.0
581586.0   DOORMAT RED RETROSPOT                  10.0
           LARGE CAKE STAND  HANGING STRAWBERY     8.0
           RED RETROSPOT ROUND CAKE TINS          24.0
           SET OF 3 HANGING OWLS OLLIE BEAK       24.0
Name: Quantity, Length: 483278, dtype: float64

In [ ]:
france_pivot_data = francedf.unstack().reset_index()

In [ ]:
france_data =france_pivot_data.fillna(0).set_index('InvoiceNo')

In [ ]:
france_data

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,20713 wrongly marked,20713.0,3 BIRDS CANVAS SCREEN,3 BLACK CATS W HEARTS BLANK CARD,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 GARDENIA MORRIS BOXED CANDLES,3 HEARTS HANGING DECORATION RUSTIC,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,...,sold as set on dotcom,sold as set on dotcom and amazon,sold as set/6 by dotcom,sold in set?,sold with wrong barcode,stock check,stock creditted wrongly,taig adjust,taig adjust no stock,temp adjustment,test,throw away,thrown away,thrown away-can't sell,thrown away-can't sell.,to push order througha s stock was,water damage,water damaged,website fixed,wet,wet boxes,wet damaged,wet pallet,wet rusty,wet/rusty,wet?,wrong barcode,wrong barcode (22467),wrong code,wrong code?,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
def hot_encoder(val):
    if val <= 0:
        return 0
    if val >=1:
        return 1
    

In [ ]:
france_data = france_data.applymap(hot_encoder)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
item_freq = apriori(france_data, min_support=.05, use_colnames=True)
item_freq

In [ ]:
rules = association_rules(item_freq, metric='lift', min_threshold=1)
rules.head()

In [ ]:
rules.sort_values(by=['lift','confidence'],ascending=[False,False],inplace= True)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
